# 1、create_sql_query_chain的使用

举例1：

In [3]:
from langchain_community.chat_models import ChatTongyi
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
import os
import dotenv
from openai.types.responses import Response

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

db_user = os.getenv("DATABASE_USER")
db_password = os.getenv("DATABASE_PASSWORD")
db_host = os.getenv("DATABASE_HOST")
db_port = os.getenv("DATABASE_PORT")
db_database = os.getenv("DATABASE_NAME")
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

chain = create_sql_query_chain(chat_model, db)
# response = chain.invoke({"question": "How many employees are there?"})
response = chain.invoke({"question": "薪资最高的员工信息？"})
print(response)

SQLQuery: SELECT `first_name`, `last_name`, `job_title`, `salary` FROM `employees` WHERE `is_active` = 1 ORDER BY `salary` DESC LIMIT 1;


# 2、create_stuff_documents_chain的使用

In [2]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
import os
import dotenv
from langchain_community.chat_models import ChatTongyi

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

prompt = PromptTemplate.from_template("""
如下文档{docs}中说，香蕉是什么颜色
""")

chain = create_stuff_documents_chain(chat_model, prompt, document_variable_name="docs")

docs = [
    Document(page_content="苹果，学名：Malus domestica，是蔷薇科苹果属的果实，通常为红色、绿色或黄色，富含维生素，膳食纤维和抗氧化物质。"),
    Document(page_content="苹果是红色的"),
    # 新增文档1：直接描述香蕉颜色
    Document(page_content="香蕉，学名：Musa acuminata，属于芭蕉科芭蕉属植物，其果实通常呈黄色，成熟后可能带有褐色斑点。"),
    # 新增文档2：通过成熟过程描述颜色变化
    Document(page_content="香蕉在未成熟时呈绿色，成熟后转变为黄色，这是由于叶绿素分解和类胡萝卜素积累的结果。"),
    # 新增文档3：不同品种的香蕉颜色差异
    Document(page_content="不同品种的香蕉颜色可能有所不同，例如红香蕉在成熟时呈红色，而普通香蕉多为黄色。"),
    # 新增文档4：农业研究视角
    Document(page_content="根据农业研究，香蕉的颜色变化与其成熟度密切相关，黄色通常表示成熟可食用。"),
    # 新增文档5：生活常识描述
    Document(page_content="日常生活中，人们常说香蕉是黄色的，这种颜色在水果摊和超市中最为常见。"),
    # 新增文档6：科学解释
    Document(page_content="香蕉的黄色主要来自类胡萝卜素和叶黄素，这些色素在果实成熟过程中逐渐积累。")
]

response = chain.invoke({"docs": docs})
print(response)

ImportError: cannot import name 'PipelinePromptTemplate' from 'langchain_core.prompts' (C:\Users\yzy\.conda\envs\langchain0.3_study_env\lib\site-packages\langchain_core\prompts\__init__.py)